In [0]:
include __DIR__ . '/vendor/autoload.php';

use Rubix\ML\Datasets\Labeled;
use Rubix\ML\Datasets\Unlabeled;
use Rubix\ML\Classifiers\GaussianNB;
use Rubix\ML\PersistentModel;
use Rubix\ML\Serializers\RBX;
use Rubix\ML\Persisters\Filesystem;
use Rubix\ML\CrossValidation\Metrics\FBeta;



In [1]:
echo date(DATE_RFC2822) . '<br>';
$data = igbinary_unserialize(file_get_contents("work/training-data.bin"));
echo date(DATE_RFC2822) . '<br>';
echo "Read " . count($data['train'][0]) . " samples";

$samples = $data['train'][0];
$labels  = $data['train'][1];

$test_samples = $data['dev'][0];
$test_labels  = $data['dev'][1];

// impute with zeros
$row_count  = count($samples);
$feat_count = count($samples[0]);
for($idx=0; $idx < $row_count; $idx++) {
    for($f=0; $f < $feat_count; $f++) {
        if(is_nan($samples[$idx][$f])) {
            $samples[$idx][$f] = 0;
        }
    }
}
$row_count  = count($test_samples);
for($idx=0; $idx < $row_count; $idx++) {
    for($f=0; $f < $feat_count; $f++) {
        if(is_nan($test_samples[$idx][$f])) {
            $test_samples[$idx][$f] = 0;
        }
    }
}
unset($data);



Sat, 11 Jun 2022 01:29:39 -0700 Sat, 11 Jun 2022 01:33:02 -0700 Read 3873579 samples

In [2]:
$dataset = new Labeled($samples, $labels);

echo date(DATE_RFC2822) . '<br>';
$estimator = new GaussianNB(NULL, 1e-9);

$estimator->train($dataset);
echo date(DATE_RFC2822) . '<br>';

$persist = new PersistentModel($estimator, new Filesystem('work/submit1.model'), new RBX());
$persist->save();


Sat, 11 Jun 2022 01:38:50 -0700 Sat, 11 Jun 2022 01:40:30 -0700

In [3]:
echo 'Testing on '.count($test_samples).' samples<br>';
$testset = new Unlabeled($test_samples);
$predictions = $estimator->predict($testset);
$metric = new FBeta();
echo "F1: " . $metric->score($predictions, $test_labels);



Testing on 548496 samples F1: 0.78989260154943